In [1]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import lightgbm as lgbm
from lightgbm import early_stopping  # Early stopping: utilizado para evitar sobreajuste

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

In [2]:
# Obter dados socioeconômicos e notas do ENEM 2023
df_enem = pd.read_pickle('Bases\Finais\dados_socioeconomicos_2023.pkl')

#Variaveis alvo
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow -> Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

# Obter colunas categóricas
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
# Apenas utilizado nos modelos fianais, BayesSearchCV não utiliza Eval Set já possui validacao cruzada interna
X_train_final, X_eval, y_train_final, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

Modelo Base

In [3]:
# Treinar modelo LGBMRegressor Base
modelo_lgbm = lgbm.LGBMRegressor(n_estimators=1000, 
                                 learning_rate=0.01, 
                                 random_state=42,
                                 max_bin=4095,
                                 force_row_wise=True)

start_time = time.time()

modelo_lgbm.fit(X_train_final, 
                y_train_final['NUM_NOTA_CH'], 
                eval_set=[(X_test, y_test['NUM_NOTA_CH'])], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features)

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 189
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 33
[LightGBM] [Info] Start training from score 527.881246


In [4]:
# Previsões
y_pred = modelo_lgbm.predict(X_test)

In [5]:
nome_experimento = 'Socioeconomicos CH 2023'

registrar_modelo(experimento=nome_experimento,
                 parametros={**modelo_lgbm.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                 X_train=X_train,
                 y_train=y_train,
                 y_test=y_test,
                 y_pred=y_pred,
                 variavel_alvo='NUM_NOTA_CH',
                 modelo=modelo_lgbm,
                 nome_modelo='modelo_lgbm_base_socioeconomicos_2023',
                 descricao_modelo='Modelo LGBMRegressor base para dados socioeconomicos CH',)

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_base_socioeconomicos_2023' already exists. Creating a new version of this model...
2025/06/14 16:04:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model versio

🏃 View run incongruous-mink-508 at: http://127.0.0.1:9080/#/experiments/419068002121547902/runs/15aa1df180c2473bb7a63e300f200488
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/419068002121547902
Modelo registrado com sucesso no MLflow: modelo_lgbm_base_socioeconomicos_2023
Rastreamento do MLflow finalizado.


Created version '2' of model 'modelo_lgbm_base_socioeconomicos_2023'.


In [6]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred, "teste")

MAE (treino): 56.6373
RMSE (treino): 71.6630
R2 (treino): 0.2834
MAE (teste): 56.6919
RMSE (teste): 71.6821
R2 (teste): 0.2792


Bayes Search

In [12]:
modelo_lgbm_bayes = lgbm.LGBMRegressor(random_state=42,
                                       max_bin=4095, 
                                       force_row_wise=True)

In [13]:
# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (5, 60),                         # Número de folhas na árvore de decisão
    'max_depth': (60, 120),                        # Profundidade máxima da árvore
    'learning_rate': (0.001, 0.01, 'log-uniform'), # Taxa de aprendizado
    'n_estimators': (5000, 8000),                  # Número de árvores
    'subsample': (0.1, 0.9),                       # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.1, 0.9),                # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-3, 1.0, 'log-uniform'),       # Regularização L1
    'reg_lambda': (1e-7, 1e-2, 'log-uniform'),     # Regularização L2
}

In [14]:
# Configurar a busca Bayesiana usando BayesSearchCV

# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_lgbm_bayes,    # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=15,                       # Número de avaliações do modelo
    cv=3,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

In [15]:
fit_params = {
    'eval_metric': ['r2', 'rmse', 'mae'],              # Métricas a serem avaliadas
    'categorical_feature': categorical_features,       # Colunas categóricas
}

In [16]:
# Executar a busca Bayesiana

start_time = time.time()
bayes_search.fit(X_train, y_train['NUM_NOTA_CH'], **fit_params)

# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[LightGBM] [Info] Total Bins 190
[LightGBM] [Info] Number of data points in the train set: 573194, number of used f

In [ ]:
# Melhores parâmetros encontrados
try:
    melhores_parametros = bayes_search.best_params_
    print(f"Melhores parâmetros: {melhores_parametros}")
    print("R2: ", bayes_search.best_score_)
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")
except:
    melhores_parametros = {
        'colsample_bytree': 0.42808316708265115, 
        'learning_rate': 0.0053422688874711095, 
        'max_depth': 116, 
        'n_estimators': 5947, 
        'num_leaves': 42, 
        'reg_alpha': 0.017472534129202036, 
        'reg_lambda': 5.684034097210144e-06, 
        'subsample': 0.6916033873523364}
    print(f"Erro ao obter melhores parâmetros, usando valores calculados anteriormente:\n {melhores_parametros}")

Melhores parâmetros: OrderedDict([('colsample_bytree', 0.42808316708265115), ('learning_rate', 0.0053422688874711095), ('max_depth', 116), ('n_estimators', 5947), ('num_leaves', 42), ('reg_alpha', 0.017472534129202036), ('reg_lambda', 5.684034097210144e-06), ('subsample', 0.6916033873523364)])
R2:  0.2810031409514067
Tempo total de execução: 81389.43 segundos


In [18]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.fit(X_train_final, 
                      y_train_final['NUM_NOTA_CH'], 
                      eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                      eval_metric=['r2', 'rmse', 'mae'],
                      categorical_feature=categorical_features,
                      callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 189
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 33
[LightGBM] [Info] Start training from score 527.881246
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[5941]	valid_0's rmse: 71.9672	valid_0's l1: 56.8267	valid_0's l2: 5179.28


In [19]:
# Previsões
y_pred_bayes = modelo_lgbm_bayes.predict(X_test)

In [20]:
nome_experimento = 'Socioeconomicos CH 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_bayes,
                    parametros={**modelo_lgbm_bayes.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_bayes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_bayes_socioeconomicos_2023',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV para dados socioeconomicos CH',)

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_bayes_socioeconomicos_2023' already exists. Creating a new version of this model...
2025/06/15 16:07:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model versi

🏃 View run grandiose-shoat-316 at: http://127.0.0.1:9080/#/experiments/419068002121547902/runs/b483e8ee8675438ea8219775e96c7df6
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/419068002121547902
Modelo registrado com sucesso no MLflow: modelo_lgbm_bayes_socioeconomicos_2023
Rastreamento do MLflow finalizado.


Created version '2' of model 'modelo_lgbm_bayes_socioeconomicos_2023'.


In [21]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_bayes.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_bayes, "teste")

MAE (treino): 56.1503
RMSE (treino): 71.0767
R2 (treino): 0.2951
MAE (teste): 56.5698
RMSE (teste): 71.5463
R2 (teste): 0.2820


In [22]:
# Salvar modelo como Pickle
joblib.dump(modelo_lgbm_bayes, 'modelos/modelo_lgbm_bayes_socioeconomico.pkl')

['modelos/modelo_lgbm_bayes_socioeconomico.pkl']